<a href="https://colab.research.google.com/github/sv2639/proj3/blob/main/proj3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 1: Explore Different Modes of Operation Through Manual Encryption and Decryption

## Problem 1: Encryption Modes

### Part 2a - Mode: ECB (Electronic Codebook)
Convert the plaintext **FOO** into binary using the provided table:
- **F** = 0101  
- **O** = 1110  
- **O** = 1110  

Apply the block cipher **Ek** to each block:
- **Ek(0101) = 1001**  
- **Ek(1110) = 1110**  
- **Ek(1110) = 1110**  

Convert the ciphertext binary back into letters using the table:
- **1001 = J**  
- **1110 = O**  

Final output:  
**FOO → JOO**  

---

### Part 2b - Mode: CBC (Cipher Block Chaining) with IV = 1010
#### Step 1: Encrypt each block  
1. **Block 1**:  
   - XOR with Initialization Vector:  
     - **0101 ⊕ 1010 = 1111**  
   - Apply the cipher **Ek** on the block:  
     - **Ek(1111) = 1111**  

2. **Block 2**:  
   - XOR with previous ciphertext:  
     - **1110 ⊕ 1111 = 0001**  
   - Apply **Ek**:  
     - **Ek(0001) = 0001**  

3. **Block 3**:  
   - XOR with previous ciphertext:  
     - **1110 ⊕ 0001 = 1111**  
   - Apply **Ek**:  
     - **Ek(1111) = 1111**  

Convert **1111 0001 1111** back into letters using the table:  
**FOO → PBP**  
The cipher text is  **KPBP**

The first position is the IV.

---

### Part 2c - Mode: CTR (Counter) with ctr = 1010
1. Encrypt the counter:  
   - **CTR = 1010**  
   - Apply **Ek**:  
     - **Ek(1010) = 0110**  

2. Encrypt each block:  
   - **Block 1**:  
     - **0101 ⊕ 0110 = 0011**  

   - **Increase counter**:  
     - **CTR = 1011**  
     - **Ek(1011) = 0111**  

   - **Block 2**:  
     - **1110 ⊕ 0111 = 1001**  

   - **Increase counter**:  
     - **CTR = 1100**  
     - **Ek(1100) = 1010**  

   - **Block 3**:  
     - **1110 ⊕ 1010 = 0100**  

Convert **0011 1001 0100** back into letters using the table:  
**FOO → DJE**  
The cipher text is  **KDJE**

The first position is the initial counter/nonce.

---

## Problem 1: Decryption Task

### 1) ECB (Electronic Codebook)
Convert the ciphertext into binary:
- **J** = 1001  
- **O** = 1110  
- **O** = 1110  

Apply the inverse of the encryption function:
- **1001 → 0101 → F**  
- **1110 → 1110 → O**  
- **1110 → 1110 → O**  

Final decrypted output:  
**JOO → FOO**  

---

### 2) CBC (Cipher Block Chaining) with IV = 1010
Convert the ciphertext to binary:
- **K** = 1010 (IV)
- **P** = 1111  
- **B** = 0001  
- **P** = 1111  

Decrypt using CBC:
1. XOR with Initialization Vector:  
   - **1010 ⊕ 1111 = 0101**  
   - Apply **Ek⁻¹**:  
     - **1001 → F**  

2. **Block 2**:  
   - XOR with previous ciphertext:  
     - **0001 ⊕ 1111 = 1110**  
   - Apply **Ek⁻¹**:  
     - **1110 → O**  

3. **Block 3**:  
   - XOR with previous ciphertext:  
     - **0001 ⊕ 1111 = 1110**  
   - Apply **Ek⁻¹**:  
     - **1110 → O**  

Final decrypted output:  
**PBP → FOO**  

---

### 3) CTR (Counter Mode) with Counter = 1010
Convert ciphertext into binary:
- **DJE** = **0011 1001 0100**  

Counters:  
- **Counter1** = **1010**  
- **Counter2** = **1011**  
- **Counter3** = **1100**  

Decrypt each block:
1. **Encrypt Counter1**:  
   - **Ek(1010) = 0110**  
   - **c1 = D = 0011**  
   - **o1 ⊕ c1 = 0110 ⊕ 0011 = 0101 (F)**  

2. **Encrypt Counter2**:  
   - **Ek(1011) = 0111**  
   - **c2 = J = 1001**  
   - **o2 ⊕ c2 = 0111 ⊕ 1001 = 1110 (O)**  

3. **Encrypt Counter3**:  
   - **Ek(1100) = 1010**  
   - **c3 = E = 0100**  
   - **o3 ⊕ c3 = 1010 ⊕ 0100 = 1110 (O)**  

Final decrypted output:  
**DJE → FOO**  


# Problem 2

## Task 1A

In [72]:
# KEY EXPANSION


import sys

# AES S-Box (from standard AES specification)
s_box = [
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
]

# Generate Inverse S-Box
inv_s_box = [0] * 256
for i in range(256):
    inv_s_box[s_box[i]] = i


Rcon = [0x21, 0x13, 0x37, 0x7F, 0xAC, 0x99]


def sub_byte(byte):
    return s_box[byte]

def inv_sub_byte(byte):
    return inv_s_box[byte]

def rot_word_16bit(word_16bit):
    byte1 = word_16bit & 0xFF  # Least significant byte
    byte2 = (word_16bit >> 8) & 0xFF # Most significant byte
    return (byte2 << 8) | byte1 # Swap bytes

def rot_word_8bit(word_8bit):
    byte1 = word_8bit & 0xF  # Least significant byte
    byte2 = (word_8bit >> 4) & 0xF # Most significant byte
    return (byte2 << 4) | byte1 # Swap bytes

def sub_word_8bit(word_8bit):
    byte1 = word_8bit & 0xF
    byte2 = (word_8bit >> 4) & 0xF
    # Apply the substitution box and make sure the result is within 8 bits
    substituted = (sub_byte(byte2) << 4) | sub_byte(byte1)

    return substituted & 0xFF  # Mask the result to 8 bits

def sub_word_16bit(word_16bit):
    byte1 = word_16bit & 0xFF
    byte2 = (word_16bit >> 8) & 0xFF
    return (sub_byte(byte2) << 8) | sub_byte(byte1)


def inv_sub_word_16bit(word_16bit):
    byte1 = word_16bit & 0xFF
    byte2 = (word_16bit >> 8) & 0xFF
    return (inv_sub_byte(byte2) << 8) | inv_sub_byte(byte1)

def key_expansion(key):
    """
    Simplified key expansion for a 16-bit key to generate 3 round keys (each 16-bit): Key0, Key1, Key2.
    key_16bit: 16-bit integer representing the AES key.
    Returns: A list of 3 round keys [Key0, Key1, Key2] (each 16-bit integer).
    """
    words = [0] * 6 # We need 6 words W0, W1, W2, W3, W4, W5, which will be Key0, Key1, Key2
    words[0] = (key >> 8) & 0xFF  # Initialize first word with the first 8-bits of the key
    words[1] = key & 0xFF   # Initialize second word with the second 8-bits of the key

    for i in range(2, 6): # Generate words W2 to W5
        temp = words[i-1]
        temp_rot = rot_word_8bit(temp) # rotate previous word
        temp_sub = sub_word_8bit(temp_rot)  #substitute previous word
        rc_val = Rcon[i-2] #constant
        words[i] = words[i-1] ^ temp_sub ^ rc_val #xor previous word with rotated+subbed word and constant

    # Combine two 8-bit words into 16-bit keys: key0, key1, key2
    key0 = (words[0] << 8) | words[1]  # Combine w0 and w1 into key0 (16 bits)
    key1 = (words[2] << 8) | words[3]  # Combine w2 and w3 into key1 (16 bits)
    key2 = (words[4] << 8) | words[5]  # Combine w4 and w5 into key2 (16 bits)

    return key0, key1, key2

example_key_16bit = 0xABCD # Example 16-bit key

round_keys = key_expansion(example_key_16bit)

print("Round Keys (16-bit hex) - Key0, Key1, Key2:")
print(f"Key0 = 0x{round_keys[0]:04X}")
print(f"Key1 = 0x{round_keys[1]:04X}")
print(f"Key2 = 0x{round_keys[2]:04X}")


Round Keys (16-bit hex) - Key0, Key1, Key2:
Key0 = 0xABCD
Key1 = 0x1BE3
Key2 = 0x2F26


In [73]:
#ENCRYPTION


def subsitute_state(state_16bit):
    return sub_word_16bit(state_16bit)

def shift_state(state_16bit):
    return rot_word_16bit(state_16bit)

def mix_state(state_16bit):
    byte1 = state_16bit & 0xFF
    byte2 = (state_16bit >> 8) & 0xFF
    mixed_byte1 = byte2
    mixed_byte2 = byte1 ^ byte2
    return (mixed_byte2 << 8) | mixed_byte1

def add_round_key(state_16bit, round_key_16bit):
    return state_16bit ^ round_key_16bit

def encrypt_round1(plaintext_16bit, key1_16bit):
    state = plaintext_16bit
    state = subsitute_state(state)
    state = shift_state(state)
    state = mix_state(state)
    state = add_round_key(state, key1_16bit)
    return state # Intermediate state X

def encrypt_round2(intermediate_state_16bit, key2_16bit):
    state = intermediate_state_16bit
    state = subsitute_state(state)
    state = shift_state(state)
    state = add_round_key(state, key2_16bit) # No Mix in the last round
    return state # Ciphertext


# Combined Encryption function
def encrypt(plaintext_16bit, key_16bit):
    """
    Encrypts a 16-bit plaintext using the simplified 2-round SAES with a 16-bit key.
    plaintext_16bit: 16-bit integer plaintext.
    key_16bit: 16-bit integer key.
    Returns: 16-bit integer ciphertext.
    """
    round_keys = key_expansion(key_16bit)
    key1, key2 = round_keys[1], round_keys[2] # Use Key1 and Key2 for encryption rounds

    intermediate_state_X = encrypt_round1(plaintext_16bit, key1)
    ciphertext_16bit = encrypt_round2(intermediate_state_X, key2)
    return ciphertext_16bit

In [74]:
#DECRYPTION


def inverse_substitute_state(state_16bit):
    return inv_sub_word_16bit(state_16bit)

def inverse_shift_state(state_16bit):
    return shift_state(state_16bit) # ShiftRows is its own inverse for byte swap

def inverse_mix_state(state_16bit):
    # If mix_state is simplified to: mixed_byte1 = byte1 ^ byte2; mixed_byte2 = byte2;
    # then inverting it: byte2 = mixed_byte2; byte1 = mixed_byte1 ^ byte2 = mixed_byte1 ^ mixed_byte2;
    mixed_byte1 = state_16bit & 0xFF
    mixed_byte2 = (state_16bit >> 8) & 0xFF
    byte2 = mixed_byte1
    byte1 = mixed_byte2 ^ byte2
    return (byte2 << 8) | byte1


def decrypt_round2(ciphertext_16bit, key2_16bit): # Decryption round 2 (first decryption round)
    state = ciphertext_16bit
    state = add_round_key(state, key2_16bit)
    state = inverse_shift_state(state)
    state = inverse_substitute_state(state)
    return state # Intermediate state Y

def decrypt_round1(intermediate_state_16bit, key1_16bit): # Decryption round 1 (second decryption round)
    state = intermediate_state_16bit
    state = add_round_key(state, key1_16bit)
    state = inverse_mix_state(state)
    state = inverse_shift_state(state)
    state = inverse_substitute_state(state)
    return state # Plaintext

# Combined Decryption function
def decrypt(ciphertext_16bit, key_16bit):
    """
    Decrypts a 16-bit ciphertext using the simplified 2-round SAES with a 16-bit key.
    ciphertext_16bit: 16-bit integer ciphertext.
    key_16bit: 16-bit integer key (same key used for encryption).
    Returns: 16-bit integer recovered plaintext.
    """
    round_keys = key_expansion(key_16bit)
    key1, key2 = round_keys[1], round_keys[2] # Use Key1 and Key2 (same keys as in encryption, but in reverse round order in decryption process)

    intermediate_state_Y = decrypt_round2(ciphertext_16bit, key2) # Decryption round 2 uses Key2
    recovered_plaintext_16bit = decrypt_round1(intermediate_state_Y, key1) # Decryption round 1 uses Key1
    return recovered_plaintext_16bit

In [75]:
import random

# Function to generate a random 16-bit key
def generate_random_key_16bit():
    """
    Generates a random 16-bit key.
    Returns: A random 16-bit integer.
    """
    return random.randint(0, 0xFFFF)

In [76]:

# Example usage with a 16-bit key and plaintext
example_key_16bit = generate_random_key_16bit()
plaintext_16bit = 0x1234 # Example 16-bit plaintext

print("Plaintext:", f"0x{plaintext_16bit:04X}")
print("Key:", f"0x{example_key_16bit:04X}")

ciphertext = encrypt(plaintext_16bit, example_key_16bit)
print("\nCiphertext:", f"0x{ciphertext:04X}")

recovered_plaintext = decrypt(ciphertext, example_key_16bit)
print("Recovered Plaintext:", f"0x{recovered_plaintext:04X}")

print("\nVerification:")
if plaintext_16bit == recovered_plaintext:
  print("\n Encryption and Decryption successful! Recovered plaintext matches original plaintext.")
else:
  print("\n Encryption and Decryption failed. Recovered plaintext does NOT match original plaintext.")

Plaintext: 0x1234
Key: 0xF099

Ciphertext: 0xABDA
Recovered Plaintext: 0x1234

Verification:

 Encryption and Decryption successful! Recovered plaintext matches original plaintext.


## Task 1B

In [84]:
# Generate a random 16-bit key
example_key_16bit = generate_random_key_16bit()

print("Using Random Key:", f"0x{example_key_16bit:04X}")

plaintext_ciphertext_pairs = {} # Dictionary to store pairs

print("\nGenerating 10 Plaintext-Ciphertext pairs with random key:")
print("---------------------------------------")

for i in range(10):
    # Generate sequential plaintexts for simplicity, you can use random plaintexts as well
    plaintext_16bit = generate_random_key_16bit()
    plaintext_hex = f"0x{plaintext_16bit:04X}"

    ciphertext = encrypt(plaintext_16bit, example_key_16bit)
    ciphertext_hex = f"0x{ciphertext:04X}"

    print(f"Pair {i+1}:")
    print(f"  Plaintext:  {plaintext_hex}")
    print(f"  Ciphertext: {ciphertext_hex}")
    print("---------------------------------------")

    # Store the pair in the dictionary
    plaintext_ciphertext_pairs[f"Pair {i+1}"] = {
        "plaintext": plaintext_hex,
        "ciphertext": ciphertext_hex
    }

print("\nVerification of Decryption for the last pair:")
last_plaintext_hex = plaintext_hex
last_ciphertext_hex = ciphertext_hex
recovered_plaintext = decrypt(int(last_ciphertext_hex, 16), example_key_16bit)

print(f"Ciphertext to decrypt: {last_ciphertext_hex}")
print(f"Recovered Plaintext:   0x{recovered_plaintext:04X}")

if int(last_plaintext_hex, 16) == recovered_plaintext:
    print("\n Decryption Verified for the last pair: Recovered plaintext matches original.")
else:
    print("\n Decryption Verification Failed for the last pair.")

print("\nPlaintext-Ciphertext Pairs Dictionary:")
print(plaintext_ciphertext_pairs)

Using Random Key: 0x4001

Generating 10 Plaintext-Ciphertext pairs with random key:
---------------------------------------
Pair 1:
  Plaintext:  0x7FC2
  Ciphertext: 0x1889
---------------------------------------
Pair 2:
  Plaintext:  0xA5BB
  Ciphertext: 0x9DDB
---------------------------------------
Pair 3:
  Plaintext:  0x713F
  Ciphertext: 0x04BB
---------------------------------------
Pair 4:
  Plaintext:  0x6414
  Ciphertext: 0xA3FB
---------------------------------------
Pair 5:
  Plaintext:  0xB053
  Ciphertext: 0xCBC3
---------------------------------------
Pair 6:
  Plaintext:  0x2F1E
  Ciphertext: 0x983B
---------------------------------------
Pair 7:
  Plaintext:  0x309B
  Ciphertext: 0x53A7
---------------------------------------
Pair 8:
  Plaintext:  0x27D4
  Ciphertext: 0x1B8D
---------------------------------------
Pair 9:
  Plaintext:  0x2C96
  Ciphertext: 0x00C8
---------------------------------------
Pair 10:
  Plaintext:  0x7EBF
  Ciphertext: 0x265E
---------------

## Task 2A

In [87]:
def mitm_attack_using_pairs(plaintext_ciphertext_pairs):
    """
    Performs a Meet-in-the-Middle attack using multiple plaintext-ciphertext pairs.
    plaintext_ciphertext_pairs: A dictionary of plaintext-ciphertext pairs.
    Returns: A set of potential key pairs (Key1, Key2).
    """
    possible_key_pairs_per_pair = []

    for pair_name, pair_data in plaintext_ciphertext_pairs.items():
        plaintext1 = int(pair_data['plaintext'], 16)
        ciphertext1 = int(pair_data['ciphertext'], 16)
        current_possible_key_pairs = set()
        print(f"\nProcessing pair: {pair_name} (Plaintext: 0x{plaintext1:04X}, Ciphertext: 0x{ciphertext1:04X})")

        #FIND X
        intermediate_values_encrypt = {}
        for key1 in range(0x10000):
            intermediate = encrypt_round1(plaintext1, key1)
            if intermediate not in intermediate_values_encrypt:
                intermediate_values_encrypt[intermediate] = []
            intermediate_values_encrypt[intermediate].append(key1)

        #FIND X'
        for key2 in range(0x10000):
            intermediate_decrypt = decrypt_round2(ciphertext1, key2)
            if intermediate_decrypt in intermediate_values_encrypt:
                for key1 in intermediate_values_encrypt[intermediate_decrypt]:
                    current_possible_key_pairs.add((key1, key2))

        print(f"Found {len(current_possible_key_pairs)} potential key pairs for this pair.")
        possible_key_pairs_per_pair.append(current_possible_key_pairs)

    # Find the intersection of all possible key pairs
    if not possible_key_pairs_per_pair:
        return set()

    final_possible_key_pairs = possible_key_pairs_per_pair[0]
    for i in range(1, len(possible_key_pairs_per_pair)):
        final_possible_key_pairs = final_possible_key_pairs.intersection(possible_key_pairs_per_pair[i])

    # Determines the number of potential key pairs
    print(f"\nNumber of potential key pairs consistent with all provided pairs: {len(final_possible_key_pairs)}")
    return final_possible_key_pairs


## Task 2B

In [91]:
#TRUE KEYS
round_keys_true = key_expansion(example_key_16bit)
true_key1 = round_keys_true[1]
true_key2 = round_keys_true[2]

# Perform MITM attack using all generated plaintext-ciphertext pairs
print("\nStarting Meet-in-the-Middle Attack using all generated pairs...")
potential_keys = mitm_attack_using_pairs(plaintext_ciphertext_pairs)

if potential_keys:
    for key1, key2 in potential_keys:
        #PRRINT TRUE KEY PAIR
        print("\nTrue Key pair:\n")
        print(f"  True Key1:      0x{true_key1:04X}")
        print(f"  True Key2:      0x{true_key2:04X}")
        #PRINT FOUND KEY PAIR
        print("\nFound Key Pair:\n")
        print(f"  Found Key1:     0x{key1:04X}")
        print(f"  Found Key2:     0x{key2:04X}")
else:
    print("\nNo matching key pairs found.")


Starting Meet-in-the-Middle Attack using all generated pairs...

Processing pair: Pair 1 (Plaintext: 0x7FC2, Ciphertext: 0x1889)
Found 65536 potential key pairs for this pair.

Processing pair: Pair 2 (Plaintext: 0xA5BB, Ciphertext: 0x9DDB)
Found 65536 potential key pairs for this pair.

Processing pair: Pair 3 (Plaintext: 0x713F, Ciphertext: 0x04BB)
Found 65536 potential key pairs for this pair.

Processing pair: Pair 4 (Plaintext: 0x6414, Ciphertext: 0xA3FB)
Found 65536 potential key pairs for this pair.

Processing pair: Pair 5 (Plaintext: 0xB053, Ciphertext: 0xCBC3)
Found 65536 potential key pairs for this pair.

Processing pair: Pair 6 (Plaintext: 0x2F1E, Ciphertext: 0x983B)
Found 65536 potential key pairs for this pair.

Processing pair: Pair 7 (Plaintext: 0x309B, Ciphertext: 0x53A7)
Found 65536 potential key pairs for this pair.

Processing pair: Pair 8 (Plaintext: 0x27D4, Ciphertext: 0x1B8D)
Found 65536 potential key pairs for this pair.

Processing pair: Pair 9 (Plaintext: 0x

## Task 3A

What is the key space for the mini block cipher?


*   The key size for the mini block cipher is 16 bits.
*   This means the keyspace is 2^16 or 65,536






## Task 3B

If the mini block cipher is executed twice (double mini block cipher), how many operations are needed for a MITM attack?


* Now, we have a 32-bit key, meaning the key space grows to 2^32.
* The MITM attack splits the key search into two parts:
  * One part encrypts all possible 2^16 values for Key1.
  * The second part decrypts all possible 2^16 values for Key2.
* Number of operations:
  * 2^16 + 2^16 = 2^17 = 131,072 operations

## Task 3C

 If we do exhaustive key search for the double mini block cipher, how many operations are needed?

* Exhaustive key search means testing all 2^32 possible keys.
* Number of operations:
  * 2^32 = 4,294,967,296 operations

## Task 3D

Trade-offs between Meet-in-the-Middle Attack and Exhaustive Key Search

The **Meet-in-the-Middle (MITM) attack** and **exhaustive key search** have distinct trade-offs in terms of speed and memory usage.

#### **1. Speed Trade-off**
- **MITM Attack:** Requires **\(2^{17}\)** operations, which is significantly faster than brute-force.
- **Exhaustive Key Search:** Requires **\(2^{32}\)** operations, making it infeasible for larger key sizes.
- **Advantage:** MITM is much **faster** than brute-force, making it more practical.

#### **2. Memory Trade-off**
- **MITM Attack:** Requires storing **\(2^{16}\)** intermediate values in a lookup table.
- **Exhaustive Key Search:** Requires almost no additional memory beyond key testing.
- **Advantage:** Brute-force requires **less memory**, whereas MITM **trades memory for speed**.

#### **3. Applicability Trade-off**
- **MITM Attack:** Only works for ciphers with a **specific structure** (e.g., two independent rounds).
- **Exhaustive Key Search:** Works for **any** cipher.
- **Advantage:** Brute-force is **more general**, but MITM is **more efficient when applicable**.

#### **Conclusion**
The **MITM attack** is significantly **faster** than exhaustive search, but it requires **additional memory** to store intermediate values. It is a **practical attack** when applicable but does not work on ciphers specifically designed to resist it (e.g., AES). On the other hand, **exhaustive search works universally but is computationally infeasible for large key sizes.**